In [1]:
import numpy as np
import mdtraj as md
import nglview as nv

# Data

In [2]:

pdb_path = f"/home/shpark/prj-mlcv/lib/DESRES/data/CLN025_desres.pdb"
traj_pdb = md.load_pdb(pdb_path)

view_pdb = nv.show_mdtraj(traj_pdb)
view_pdb


view_pdb.clear_representations()
# view_pdb.add_representation("ball+stick", selection="not hydrogen")
# view_pdb.add_representation("licorice", selection="not hydrogen")
view_pdb.add_representation("licorice", selection="protein")
view_pdb

NGLWidget()

# Samples

In [3]:
date = "0729_091523/1bond"

xtc_path = f"/home/shpark/prj-mlcv/lib/bioemu/res/cln025/{date}/samples.xtc"
pdb_path = f"/home/shpark/prj-mlcv/lib/bioemu/res/cln025/{date}/topology.pdb"
reconstructed_xtc_path = f"/home/shpark/prj-mlcv/lib/bioemu/res/cln025/{date}/samples_sidechain_rec.xtc"
reconstructed_pdb_path = f"/home/shpark/prj-mlcv/lib/bioemu/res/cln025/{date}/samples_sidechain_rec.pdb"

traj = md.load(reconstructed_xtc_path, top=reconstructed_pdb_path)
print(traj)

view = nv.show_mdtraj(traj)
view.add_representation("licorice", selection="protein")
# view.add_representation("licorice", selection="not hydrogen")
view

<mdtraj.Trajectory with 100 frames, 92 atoms, 10 residues, without unitcells>


NGLWidget(max_frame=99)

In [9]:
xtc_path = f"/home/shpark/prj-mlcv/lib/bioemu/res/cln025/{date}/samples_md_equil.xtc"
pdb_path = f"/home/shpark/prj-mlcv/lib/bioemu/res/cln025/{date}/samples_md_equil.pdb"

traj = md.load(xtc_path, top=pdb_path)
print(traj)

view = nv.show_mdtraj(traj)
view.add_representation("licorice", selection="protein")
view

<mdtraj.Trajectory with 100 frames, 93 atoms, 10 residues, without unitcells>


NGLWidget(max_frame=99)

In [10]:
view.frame = 25

In [ ]:
condition_pdb = "/home/shpark/prj-mlcv/lib/DESRES/data/6bond.pdb"

condition_pdb_traj = md.load_pdb(condition_pdb)
condition_pdb_view = nv.show_mdtraj(condition_pdb_traj)
condition_pdb_view.add_representation("licorice", selection="protein")
condition_pdb_view

NGLWidget()